# Importando bibliotecas

In [1]:
from pyspark.sql import SparkSession

# Criando sessão Spark

In [2]:
spark = SparkSession.builder.master("local[*]").appName("Classificação com Spark").getOrCreate()

In [3]:
spark

# Carregando os dados

In [4]:
dados = spark.read.csv("datasets/dados_clientes.csv", sep=",", header=True, inferSchema=True)

In [5]:
dados.show()

+---+-----+----------+-------+-----------+---------------+------------+------------------------+-----------+------------------+------------------+------------------+------------------+------------------+------------------+------------+------------+----------------+-------------+
| id|Churn|Mais65anos|Conjuge|Dependentes|MesesDeContrato|TelefoneFixo|MaisDeUmaLinhaTelefonica|   Internet|   SegurancaOnline|      BackupOnline| SeguroDispositivo|    SuporteTecnico|           TVaCabo|   StreamingFilmes|TipoContrato|ContaCorreio| MetodoPagamento|MesesCobrados|
+---+-----+----------+-------+-----------+---------------+------------+------------------------+-----------+------------------+------------------+------------------+------------------+------------------+------------------+------------+------------+----------------+-------------+
|  0|  Nao|         0|    Sim|        Nao|              1|         Nao|    SemServicoTelefonico|        DSL|               Nao|               Sim|              

In [6]:
dados.count()

10348

In [7]:
dados.groupBy("Churn").count().show()

+-----+-----+
|Churn|count|
+-----+-----+
|  Sim| 5174|
|  Nao| 5174|
+-----+-----+



In [8]:
dados.printSchema()

root
 |-- id: integer (nullable = true)
 |-- Churn: string (nullable = true)
 |-- Mais65anos: integer (nullable = true)
 |-- Conjuge: string (nullable = true)
 |-- Dependentes: string (nullable = true)
 |-- MesesDeContrato: integer (nullable = true)
 |-- TelefoneFixo: string (nullable = true)
 |-- MaisDeUmaLinhaTelefonica: string (nullable = true)
 |-- Internet: string (nullable = true)
 |-- SegurancaOnline: string (nullable = true)
 |-- BackupOnline: string (nullable = true)
 |-- SeguroDispositivo: string (nullable = true)
 |-- SuporteTecnico: string (nullable = true)
 |-- TVaCabo: string (nullable = true)
 |-- StreamingFilmes: string (nullable = true)
 |-- TipoContrato: string (nullable = true)
 |-- ContaCorreio: string (nullable = true)
 |-- MetodoPagamento: string (nullable = true)
 |-- MesesCobrados: double (nullable = true)



# Tratando os dados

In [9]:
colunasBinarias = [
    "Churn",
    "Conjuge",
    "Dependentes",
    "TelefoneFixo",
    "MaisDeUmaLinhaTelefonica",
    "SegurancaOnline",
    "BackupOnline",
    "SeguroDispositivo",
    "SuporteTecnico",
    "TVaCabo",
    "StreamingFilmes",
    "ContaCorreio"
]

In [10]:
from pyspark.sql import functions as f

In [11]:
todasColunas = [f.when(f.col(col) == "Sim", 1).otherwise(0).alias(col) for col in colunasBinarias]

In [12]:
for coluna in reversed(dados.columns):
    if coluna not in colunasBinarias:
        todasColunas.insert(0, coluna)
todasColunas

['id',
 'Mais65anos',
 'MesesDeContrato',
 'Internet',
 'TipoContrato',
 'MetodoPagamento',
 'MesesCobrados',
 Column<'CASE WHEN (Churn = Sim) THEN 1 ELSE 0 END AS Churn'>,
 Column<'CASE WHEN (Conjuge = Sim) THEN 1 ELSE 0 END AS Conjuge'>,
 Column<'CASE WHEN (Dependentes = Sim) THEN 1 ELSE 0 END AS Dependentes'>,
 Column<'CASE WHEN (TelefoneFixo = Sim) THEN 1 ELSE 0 END AS TelefoneFixo'>,
 Column<'CASE WHEN (MaisDeUmaLinhaTelefonica = Sim) THEN 1 ELSE 0 END AS MaisDeUmaLinhaTelefonica'>,
 Column<'CASE WHEN (SegurancaOnline = Sim) THEN 1 ELSE 0 END AS SegurancaOnline'>,
 Column<'CASE WHEN (BackupOnline = Sim) THEN 1 ELSE 0 END AS BackupOnline'>,
 Column<'CASE WHEN (SeguroDispositivo = Sim) THEN 1 ELSE 0 END AS SeguroDispositivo'>,
 Column<'CASE WHEN (SuporteTecnico = Sim) THEN 1 ELSE 0 END AS SuporteTecnico'>,
 Column<'CASE WHEN (TVaCabo = Sim) THEN 1 ELSE 0 END AS TVaCabo'>,
 Column<'CASE WHEN (StreamingFilmes = Sim) THEN 1 ELSE 0 END AS StreamingFilmes'>,
 Column<'CASE WHEN (ContaCorr

In [13]:
dados.select(todasColunas).show()

+---+----------+---------------+-----------+------------+----------------+-------------+-----+-------+-----------+------------+------------------------+---------------+------------+-----------------+--------------+-------+---------------+------------+
| id|Mais65anos|MesesDeContrato|   Internet|TipoContrato| MetodoPagamento|MesesCobrados|Churn|Conjuge|Dependentes|TelefoneFixo|MaisDeUmaLinhaTelefonica|SegurancaOnline|BackupOnline|SeguroDispositivo|SuporteTecnico|TVaCabo|StreamingFilmes|ContaCorreio|
+---+----------+---------------+-----------+------------+----------------+-------------+-----+-------+-----------+------------+------------------------+---------------+------------+-----------------+--------------+-------+---------------+------------+
|  0|         0|              1|        DSL| Mensalmente|BoletoEletronico|        29.85|    0|      1|          0|           0|                       0|              0|           1|                0|             0|      0|              0|      

In [14]:
dataset = dados.select(todasColunas)

In [15]:
dataset.printSchema()

root
 |-- id: integer (nullable = true)
 |-- Mais65anos: integer (nullable = true)
 |-- MesesDeContrato: integer (nullable = true)
 |-- Internet: string (nullable = true)
 |-- TipoContrato: string (nullable = true)
 |-- MetodoPagamento: string (nullable = true)
 |-- MesesCobrados: double (nullable = true)
 |-- Churn: integer (nullable = false)
 |-- Conjuge: integer (nullable = false)
 |-- Dependentes: integer (nullable = false)
 |-- TelefoneFixo: integer (nullable = false)
 |-- MaisDeUmaLinhaTelefonica: integer (nullable = false)
 |-- SegurancaOnline: integer (nullable = false)
 |-- BackupOnline: integer (nullable = false)
 |-- SeguroDispositivo: integer (nullable = false)
 |-- SuporteTecnico: integer (nullable = false)
 |-- TVaCabo: integer (nullable = false)
 |-- StreamingFilmes: integer (nullable = false)
 |-- ContaCorreio: integer (nullable = false)



# Criando Dummies

In [16]:
dados.select(["Internet", "TipoContrato", "MetodoPagamento"]).show()

+-----------+------------+----------------+
|   Internet|TipoContrato| MetodoPagamento|
+-----------+------------+----------------+
|        DSL| Mensalmente|BoletoEletronico|
|        DSL|       UmAno|          Boleto|
|        DSL| Mensalmente|          Boleto|
|        DSL|       UmAno|   DebitoEmConta|
|FibraOptica| Mensalmente|BoletoEletronico|
|FibraOptica| Mensalmente|BoletoEletronico|
|FibraOptica| Mensalmente|   CartaoCredito|
|        DSL| Mensalmente|          Boleto|
|FibraOptica| Mensalmente|BoletoEletronico|
|        DSL|       UmAno|   DebitoEmConta|
|        DSL| Mensalmente|          Boleto|
|        Nao|    DoisAnos|   CartaoCredito|
|FibraOptica|       UmAno|   CartaoCredito|
|FibraOptica| Mensalmente|   DebitoEmConta|
|FibraOptica| Mensalmente|BoletoEletronico|
|FibraOptica|    DoisAnos|   CartaoCredito|
|        Nao|       UmAno|          Boleto|
|FibraOptica|    DoisAnos|   DebitoEmConta|
|        DSL| Mensalmente|   CartaoCredito|
|FibraOptica| Mensalmente|Boleto

In [17]:
dataset.groupBy("id").pivot("Internet").agg(f.lit(1)).na.fill(0).show()

+----+---+-----------+---+
|  id|DSL|FibraOptica|Nao|
+----+---+-----------+---+
|7982|  1|          0|  0|
|9465|  0|          1|  0|
|2122|  1|          0|  0|
|3997|  1|          0|  0|
|6654|  0|          1|  0|
|7880|  0|          1|  0|
|4519|  0|          1|  0|
|6466|  0|          1|  0|
| 496|  1|          0|  0|
|7833|  0|          1|  0|
|1591|  0|          0|  1|
|2866|  0|          1|  0|
|8592|  0|          1|  0|
|1829|  0|          1|  0|
| 463|  0|          1|  0|
|4900|  0|          1|  0|
|4818|  0|          1|  0|
|7554|  1|          0|  0|
|1342|  0|          0|  1|
|5300|  0|          1|  0|
+----+---+-----------+---+
only showing top 20 rows



In [18]:
Internet = dataset.groupBy("id").pivot("Internet").agg(f.lit(1)).na.fill(0)
TipoContrato = dataset.groupBy("id").pivot("TipoContrato").agg(f.lit(1)).na.fill(0)
MetodoPagamento = dataset.groupBy("id").pivot("MetodoPagamento").agg(f.lit(1)).na.fill(0)

In [19]:
dataset\
    .join(Internet, "id", how="inner")\
    .join(TipoContrato, "id", how="inner")\
    .join(MetodoPagamento, "id", how="inner")\
    .select(
        "*",
        f.col("DSL").alias("Internet_DSL"),
        f.col("FibraOptica").alias("Internet_FibraOptica"),
        f.col("Nao").alias("Internet_Nao"),
        f.col("Mensalmente").alias("TipoContrato_Mensalmente"),
        f.col("UmAno").alias("TipoContrato_UmAno"),
        f.col("DoisAnos").alias("TipoContrato_DoisAnos"),
        f.col("DSL").alias("Internet_DSL"),
        f.col("DebitoEmConta").alias("MetodoPagamento_DebitoEmConta"),
        f.col("CartaoCredito").alias("MetodoPagamento_CartaoCredito"),
        f.col("BoletoEletronico").alias("MetodoPagamento_BoletoEletronico"),
        f.col("Boleto").alias("MetodoPagamento_Boleto")
    )\
        .drop(
            "Internet", "TipoContrato", "MetodoPagamento", "DSL",
            "FibraOptica", "Nao", "Mensalmente", "UmAno", "DoisAnos",
            "DebitoEmConta", "CartaoCredito", "BoletoEletronico", "Boleto"
        )\
        .show(truncate=False)
    

+----+----------+---------------+-----------------+-----+-------+-----------+------------+------------------------+---------------+------------+-----------------+--------------+-------+---------------+------------+------------+--------------------+------------+------------------------+------------------+---------------------+------------+-----------------------------+-----------------------------+--------------------------------+----------------------+
|id  |Mais65anos|MesesDeContrato|MesesCobrados    |Churn|Conjuge|Dependentes|TelefoneFixo|MaisDeUmaLinhaTelefonica|SegurancaOnline|BackupOnline|SeguroDispositivo|SuporteTecnico|TVaCabo|StreamingFilmes|ContaCorreio|Internet_DSL|Internet_FibraOptica|Internet_Nao|TipoContrato_Mensalmente|TipoContrato_UmAno|TipoContrato_DoisAnos|Internet_DSL|MetodoPagamento_DebitoEmConta|MetodoPagamento_CartaoCredito|MetodoPagamento_BoletoEletronico|MetodoPagamento_Boleto|
+----+----------+---------------+-----------------+-----+-------+-----------+---------

In [20]:
dataset = dataset\
    .join(Internet, "id", how="inner")\
    .join(TipoContrato, "id", how="inner")\
    .join(MetodoPagamento, "id", how="inner")\
    .select(
        "*",
        f.col("DSL").alias("Internet_DSL"),
        f.col("FibraOptica").alias("Internet_FibraOptica"),
        f.col("Nao").alias("Internet_Nao"),
        f.col("Mensalmente").alias("TipoContrato_Mensalmente"),
        f.col("UmAno").alias("TipoContrato_UmAno"),
        f.col("DoisAnos").alias("TipoContrato_DoisAnos"),
        f.col("DebitoEmConta").alias("MetodoPagamento_DebitoEmConta"),
        f.col("CartaoCredito").alias("MetodoPagamento_CartaoCredito"),
        f.col("BoletoEletronico").alias("MetodoPagamento_BoletoEletronico"),
        f.col("Boleto").alias("MetodoPagamento_Boleto")
    )\
        .drop(
            "Internet", "TipoContrato", "MetodoPagamento", "DSL",
            "FibraOptica", "Nao", "Mensalmente", "UmAno", "DoisAnos",
            "DebitoEmConta", "CartaoCredito", "BoletoEletronico", "Boleto"
        )

In [21]:
dataset.printSchema()

root
 |-- id: integer (nullable = true)
 |-- Mais65anos: integer (nullable = true)
 |-- MesesDeContrato: integer (nullable = true)
 |-- MesesCobrados: double (nullable = true)
 |-- Churn: integer (nullable = false)
 |-- Conjuge: integer (nullable = false)
 |-- Dependentes: integer (nullable = false)
 |-- TelefoneFixo: integer (nullable = false)
 |-- MaisDeUmaLinhaTelefonica: integer (nullable = false)
 |-- SegurancaOnline: integer (nullable = false)
 |-- BackupOnline: integer (nullable = false)
 |-- SeguroDispositivo: integer (nullable = false)
 |-- SuporteTecnico: integer (nullable = false)
 |-- TVaCabo: integer (nullable = false)
 |-- StreamingFilmes: integer (nullable = false)
 |-- ContaCorreio: integer (nullable = false)
 |-- Internet_DSL: integer (nullable = true)
 |-- Internet_FibraOptica: integer (nullable = true)
 |-- Internet_Nao: integer (nullable = true)
 |-- TipoContrato_Mensalmente: integer (nullable = true)
 |-- TipoContrato_UmAno: integer (nullable = true)
 |-- TipoContr

# O modelo

## Preparação dos Dados

In [22]:
from pyspark.ml.feature import VectorAssembler

In [23]:
dataset = dataset.withColumnRenamed("Churn", "label")

In [24]:
X = dataset.columns
X.remove("label")
X.remove("id")
X

['Mais65anos',
 'MesesDeContrato',
 'MesesCobrados',
 'Conjuge',
 'Dependentes',
 'TelefoneFixo',
 'MaisDeUmaLinhaTelefonica',
 'SegurancaOnline',
 'BackupOnline',
 'SeguroDispositivo',
 'SuporteTecnico',
 'TVaCabo',
 'StreamingFilmes',
 'ContaCorreio',
 'Internet_DSL',
 'Internet_FibraOptica',
 'Internet_Nao',
 'TipoContrato_Mensalmente',
 'TipoContrato_UmAno',
 'TipoContrato_DoisAnos',
 'MetodoPagamento_DebitoEmConta',
 'MetodoPagamento_CartaoCredito',
 'MetodoPagamento_BoletoEletronico',
 'MetodoPagamento_Boleto']

In [25]:
assembler = VectorAssembler(inputCols=X, outputCol="features")

In [26]:
dataset_prep = assembler.transform(dataset=dataset).select("features", "label")

In [27]:
dataset_prep.show(10, False)

+-----------------------------------------------------------------------------------------------------------+-----+
|features                                                                                                   |label|
+-----------------------------------------------------------------------------------------------------------+-----+
|(24,[1,2,11,12,13,14,17,22],[1.0,45.30540797610398,1.0,1.0,1.0,1.0,1.0,1.0])                               |1    |
|(24,[1,2,3,5,6,8,9,11,12,13,15,17,22],[60.0,103.6142230120257,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|1    |
|(24,[1,2,5,6,10,11,12,13,14,18,23],[12.0,75.85,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])                       |0    |
|(24,[1,2,3,5,8,12,13,14,19,21],[69.0,61.45,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])                               |0    |
|(24,[1,2,3,5,6,11,13,15,17,22],[7.0,86.5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])                                 |1    |
|(24,[1,2,5,6,12,13,15,17,22],[14.0,85.03742670311915,1.0,1.0,1.0,1.0,1.

# Ajuste e Previsão

In [28]:
SEED = 101

In [29]:
treino, teste = dataset_prep.randomSplit([0.7, 0.3], seed=SEED)

In [30]:
treino.count()

7206

In [31]:
teste.count()

3142

In [32]:
from pyspark.ml.classification import LogisticRegression

In [33]:
lr = LogisticRegression()

In [34]:
modelo_lr = lr.fit(treino)

In [35]:
previsoes_lr_teste = modelo_lr.transform(teste)

In [36]:
previsoes_lr_teste.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(24,[0,1,2,3,4,5,...|    0|[3.02174179751551...|[0.95354674000282...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[-0.0922192966076...|[0.47696150091605...|       1.0|
|(24,[0,1,2,3,4,5,...|    1|[0.18744121711361...|[0.54672358463156...|       0.0|
|(24,[0,1,2,3,4,5,...|    1|[0.91716501260103...|[0.71446410549163...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[-0.1495904711610...|[0.46267196467801...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[-0.1680594619286...|[0.45808374494006...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[-1.4170949608173...|[0.19511740608882...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[0.14194260698794...|[0.53542619200881...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[0.67046644011599...|[0.66160759507905...|       0.0|
|(24,[0,1,2,3,4,

# Métricas

In [37]:
resumo_lr_treino = modelo_lr.summary

In [38]:
print(f"Acurácia: {resumo_lr_treino.accuracy:.2f}")
print(f"Precisão: {resumo_lr_treino.precisionByLabel[1]:.2f}")
print(f"Recall: {resumo_lr_treino.recallByLabel[1]:.2f}")
print(f"F1: {resumo_lr_treino.fMeasureByLabel()[1]:.2f}")

Acurácia: 0.78
Precisão: 0.77
Recall: 0.81
F1: 0.79


In [39]:
previsoes_lr_teste.select("label", "prediction").where((f.col("label") == 1) & (f.col("prediction") == 1)).count()

1256

In [40]:
tp = previsoes_lr_teste.select("label", "prediction").where((f.col("label") == 1) & (f.col("prediction") == 1)).count()
tn = previsoes_lr_teste.select("label", "prediction").where((f.col("label") == 0) & (f.col("prediction") == 0)).count()
fp = previsoes_lr_teste.select("label", "prediction").where((f.col("label") == 0) & (f.col("prediction") == 1)).count()
fn = previsoes_lr_teste.select("label", "prediction").where((f.col("label") == 1) & (f.col("prediction") == 0)).count()
print(tp, tn, fp, fn)

1256 1179 400 307


In [41]:
def calcula_mostra_matriz_confusao(df_transform_modelo, normalize=False, percentage=True):

    tp = previsoes_lr_teste.select("label", "prediction").where((f.col("label") == 1) & (f.col("prediction") == 1)).count()
    tn = previsoes_lr_teste.select("label", "prediction").where((f.col("label") == 0) & (f.col("prediction") == 0)).count()
    fp = previsoes_lr_teste.select("label", "prediction").where((f.col("label") == 0) & (f.col("prediction") == 1)).count()
    fn = previsoes_lr_teste.select("label", "prediction").where((f.col("label") == 1) & (f.col("prediction") == 0)).count()

    valorP = 1
    valorN = 1

    if normalize:
        valorP = tp + fn
        valorN = fp + tn 

    if percentage and normalize:
        valorP = valorP / 100
        valorN = valorN / 100

    
    print(" " * 20, "Previsto")
    print(" " * 15, "Churn", " " * 5, "Não-Churn")
    print(" " * 4, "Churn", " " * 6, int(tp / valorP), " " * 7, int(fn / valorP))
    print("Real")
    print(" " * 4, "Não-Churn", " " * 2, int(fp / valorN), " " * 7, int(tn / valorN))


In [42]:
calcula_mostra_matriz_confusao(previsoes_lr_teste)

                     Previsto
                Churn       Não-Churn
     Churn        1256         307
Real
     Não-Churn    400         1179


# Árvore de Decisão
## Decision Tree

In [43]:
from pyspark.ml.classification import DecisionTreeClassifier

In [44]:
dtc = DecisionTreeClassifier(seed=SEED)

In [45]:
modelo_dtc = dtc.fit(treino)

In [46]:
previsoes_dtc_treino = modelo_dtc.transform(treino)

In [47]:
previsoes_dtc_treino.show()

+--------------------+-----+--------------+--------------------+----------+
|            features|label| rawPrediction|         probability|prediction|
+--------------------+-----+--------------+--------------------+----------+
|(24,[0,1,2,3,4,5,...|    0|[2056.0,334.0]|[0.86025104602510...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[2056.0,334.0]|[0.86025104602510...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|    [22.0,3.0]|         [0.88,0.12]|       0.0|
|(24,[0,1,2,3,4,5,...|    0|    [22.0,3.0]|         [0.88,0.12]|       0.0|
|(24,[0,1,2,3,4,5,...|    0|    [22.0,3.0]|         [0.88,0.12]|       0.0|
|(24,[0,1,2,3,4,5,...|    1|[331.0,1951.0]|[0.14504820333041...|       1.0|
|(24,[0,1,2,3,4,5,...|    0| [239.0,205.0]|[0.53828828828828...|       0.0|
|(24,[0,1,2,3,4,5,...|    1|[331.0,1951.0]|[0.14504820333041...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[331.0,1951.0]|[0.14504820333041...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[331.0,1951.0]|[0.14504820333041...|       1.0|
|(24,[0,1,2,

# Métricas

In [51]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [52]:
evaluator = MulticlassClassificationEvaluator()

In [53]:
evaluator.evaluate(previsoes_dtc_treino, {evaluator.metricName: 'accuracy'})

0.7917013599777962

In [54]:
print(f"Acurácia: {evaluator.evaluate(previsoes_dtc_treino, {evaluator.metricName: 'accuracy'})}")
print(f"Precisão: {evaluator.evaluate(previsoes_dtc_treino, {evaluator.metricName: 'precisionByLabel'})}")
print(f"Recall: {evaluator.evaluate(previsoes_dtc_treino, {evaluator.metricName: 'recallByLabel'})}")
print(f"F1: {evaluator.evaluate(previsoes_dtc_treino, {evaluator.metricName: 'fMeasureByLabel'})}")

Acurácia: 0.7917013599777962
Precisão: 0.7793489861259338
Recall: 0.8125173852573018
F1: 0.7955876344818194


In [55]:
previsoes_dtc_teste = modelo_dtc.transform(teste)

In [56]:
previsoes_dtc_teste.show()

+--------------------+-----+--------------+--------------------+----------+
|            features|label| rawPrediction|         probability|prediction|
+--------------------+-----+--------------+--------------------+----------+
|(24,[0,1,2,3,4,5,...|    0|[2056.0,334.0]|[0.86025104602510...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|  [62.0,128.0]|[0.32631578947368...|       1.0|
|(24,[0,1,2,3,4,5,...|    1| [239.0,205.0]|[0.53828828828828...|       0.0|
|(24,[0,1,2,3,4,5,...|    1| [239.0,205.0]|[0.53828828828828...|       0.0|
|(24,[0,1,2,3,4,5,...|    0| [239.0,205.0]|[0.53828828828828...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|  [51.0,141.0]| [0.265625,0.734375]|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[331.0,1951.0]|[0.14504820333041...|       1.0|
|(24,[0,1,2,3,4,5,...|    0| [239.0,205.0]|[0.53828828828828...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|  [63.0,118.0]|[0.34806629834254...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[2056.0,334.0]|[0.86025104602510...|       0.0|
|(24,[0,1,2,

In [58]:
print(f"Acurácia: {evaluator.evaluate(previsoes_dtc_teste, {evaluator.metricName: 'accuracy'})}")
print(f"Precisão: {evaluator.evaluate(previsoes_dtc_teste, {evaluator.metricName: 'precisionByLabel'})}")
print(f"Recall: {evaluator.evaluate(previsoes_dtc_teste, {evaluator.metricName: 'recallByLabel'})}")
print(f"F1: {evaluator.evaluate(previsoes_dtc_teste, {evaluator.metricName: 'fMeasureByLabel'})}")

Acurácia: 0.7714831317632082
Precisão: 0.7649230769230769
Recall: 0.7872070930968967
F1: 0.7759051186017478
